# <font color='EC2C04'>  SPAM DETECTION SYSTEM with Ling-Spam COLLECTION </font>

This notebook includes the tasks performed in the **second part** of the Resit Coursework of the Big Data Applications module. For a complete description of the purposes of the analysis, please consult the PDF document provided which also represent the first part of the Coursework.
This script goes through the entire process that is necessary to create and obtain the optimal model able to identify whether a message represents a Spam or Ham.
In particular, the phases in which the document is organised are summarised as following:

1. **Access and Import the File**  



In [2]:
# PYSPARK INSTALLATION
!pip install pyspark 

    100% |████████████████████████████████| 215.6MB 156kB/s ta 0:00:011    84% |███████████████████████████     | 182.1MB 11.6MB/s eta 0:00:03
    100% |████████████████████████████████| 204kB 7.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyspark ... done
  Stored in directory: /Users/simonezanetti/Library/Caches/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [146]:
# LOAD THE PACKAGES 
import pyspark
from pathlib import Path
import re
import nltk
import re
from nltk.corpus import stopwords

In [147]:
# DEFINE THE CONTEXT
sc = pyspark.SparkContext.getOrCreate()

In [4]:
#
cd /Users/simonezanetti/Desktop/lingspam_public


/Users/simonezanetti/Desktop/lingspam_public


In [5]:
!cat readme.txt

This directory contains the Ling-Spam corpus, as described in the 
paper:

I. Androutsopoulos, J. Koutsias, K.V. Chandrinos, George Paliouras, 
and C.D. Spyropoulos, "An Evaluation of Naive Bayesian Anti-Spam 
Filtering". In Potamias, G., Moustakis, V. and van Someren, M. (Eds.), 
Proceedings of the Workshop on Machine Learning in the New Information 
Age, 11th European Conference on Machine Learning (ECML 2000), 
Barcelona, Spain, pp. 9-17, 2000.

There are four subdirectories, corresponding to four versions of 
the corpus:

bare: Lemmatiser disabled, stop-list disabled.
lemm: Lemmatiser enabled, stop-list disabled.
lemm_stop: Lemmatiser enabled, stop-list enabled.
stop: Lemmatiser disabled, stop-list enabled.

Each one of these 4 directories contains 10 subdirectories (part1, 
..., part10). These correspond to the 10 partitions of the corpus 
that were used in the 10-fold experiments. In each repetition, one 
part was reserved for testing and the other 9 were used for training. 

Eac

In [6]:
#ssh szane001@dsm10.doc.gold.ac.uk
#cd BigDataResit/lingspam_public
#pyspark --master yarn
#import pyspark
#sc = pyspark.SparkContext.getOrCreate()

## Task 1) Read the dataset and create RDDs 

1) The first step is aimed to 


a) Start by reading the directory with text files from the file system (`/content/drive/My Drive/BigData/data/lingspam_public`). Load all text files per dirctory (part1,part2, ... ,part10) using `wholeTextFiles()`, which creates one RDD per part, containing tuples (filename,text). This is a good choice as the text files are small. (5%)

b) We will use one of the RDDs as test set, the rest as training set. For the training set you need to create the union of the remaining RDDs. (5%)

b) Remove the path and extension from the filename using the regular expression provided (5%).

If the filename starts with 'spmsg' it is spam, otherwise it is not. We'll use that later to train a classifier. 

We will put the code in each cell into a function that we can reuse later. In this way we can develop the whole preprocessing with the smaller test set and apply it to the training set once we know that everything works. 

In [9]:
# EXTRACT THE TEXT FILES AND STORE THEM INTO RDD FILES

# Goal is Extracting the text files from one of the subdirectories of the folder lingspam_public, by iterating and storing the
# content of each of the directory created ( set in the code below ) as RDD file. Find the article that says it's the best choice
# when files are small

p = Path('bare')  # Define the Path that in this case is represented by the folder Bare in the lingspam_public folder/directory
print(*p.iterdir()) # Check that this is going to do what you want -> Obtaining as directories all the parts within the folder
                    # You will after iterate over all of them to assign their content into an RDD file.
dirs = list(p.iterdir()) # Store each directory path in a list, in order to iterate that and be able to obtain the RDD's. 
# Set the loop that allows to store the content of each directory as Rdd within a RddList.
rddList = []
for d in dirs:
    # iterate through the directories
    rdd= sc.wholeTextFiles(str(d.resolve()))
    rddList.append(rdd)
# Check you have 10 directories in the list
len(rddList)
# Check that it worked 
print(rddList[1].take(1))

bare/part3 bare/part4 bare/part5 bare/part2 bare/part10 bare/part9 bare/part7 bare/part1 bare/part6 bare/part8
[('file:/Users/simonezanetti/Desktop/lingspam_public/bare/part4/6-266msg3.txt', 'Subject: bisfai deadline extension !\n\nbisfai deadline extension ! the deadline for the bar - ilan symposium on foundations of artificial intelligence has been extended to february 27 . the conference itself will take place as scheduled , june 20-22 , in ramat - gan and jerusalem , israel . for more information contact : bisfai @ bimacs . cs . biu . ac . il daniel radzinski tovna translation machines jerusalem , israel dr @ tovna . co . il\n')]


In [10]:
# SPLIT TRAIN AND TEST DATA 

# Define your Test Data
testRDD1 = rddList[9]


trainRDD1 = sc.emptyRDD() # Set an empty Rdd and Unite all the Rdd's left for Training together 
for i in range(0,9):
        trainRDD1 = trainRDD1.union(rddList[i])

# Check how the content of the RDD looks like 
trainRDD1.take(1)

[('file:/Users/simonezanetti/Desktop/lingspam_public/bare/part3/6-110msg3.txt',
  "Subject: job posting - apple-iss research center\n\ncontent - length : 3386 apple-iss research center a us $ 10 million joint venture between apple computer inc . and the institute of systems science of the national university of singapore , located in singapore , is looking for : a senior speech scientist - - - - - - - - - - - - - - - - - - - - - - - - - the successful candidate will have research expertise in computational linguistics , including natural language processing and * * english * * and * * chinese * * statistical language modeling . knowledge of state-of - the-art corpus-based n - gram language models , cache language models , and part-of - speech language models are required . a text - to - speech project leader - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - the successful candidate will have research expertise expertise in two or more of the following areas : computational 

In [12]:
# Each file still contains the path specification, which at this point is useless.
# This will be removed using 

testRDD = testRDD1.map(lambda x: (re.split('[/\.]', x[0])[-2], x[1]))  # <- CHECK THIS. The less clear to me 
trainRDD = trainRDD1.map(lambda x: (re.split('[/\.]', x[0])[-2], x[1]))

In [14]:
# Check the length of both the files and have a look 

print('testRDD.count(): ',testRDD2.count()) # should be ~291 
#print('trainRDD.count(): ',trainRDD.count()) # should be ~2602 - commented out to save time as it takes some time to create RDD from all the files

trainRDD.take(2)

testRDD.count():  289


[('6-110msg3',
  "Subject: job posting - apple-iss research center\n\ncontent - length : 3386 apple-iss research center a us $ 10 million joint venture between apple computer inc . and the institute of systems science of the national university of singapore , located in singapore , is looking for : a senior speech scientist - - - - - - - - - - - - - - - - - - - - - - - - - the successful candidate will have research expertise in computational linguistics , including natural language processing and * * english * * and * * chinese * * statistical language modeling . knowledge of state-of - the-art corpus-based n - gram language models , cache language models , and part-of - speech language models are required . a text - to - speech project leader - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - the successful candidate will have research expertise expertise in two or more of the following areas : computational linguistics , including natural language parsing , lexical databa

In [16]:
print('testRDD.getNumPartitions()',testRDD2.getNumPartitions()) # normally 2 on Colab (single machine)
print('testRDD.getStorageLevel()',testRDD2.getStorageLevel()) # Serialized, 1x Replicated 
print('testRDD.take(1): ',testRDD2.take(3)) # should be (filename,[tokens]) 
rdd1 = testRDD # use this for developemnt in the next tasks 

testRDD.getNumPartitions() 2
testRDD.getStorageLevel() Serialized 1x Replicated
testRDD.take(1):  [('8-1064msg1', 'Subject: re : 8 . 1044 , disc : grammar in schools\n\n( re message from : linguist @ linguistlist . org ) > > linguist list : vol-8 - 1044 . sat jul 12 1997 . issn : 1068-4875 . > > subject : 8 . 1044 , disc : grammar in schools > > i know and teach that not all infinitives contain ` to \' . i also give > the students examples ( e . g . ` i asked him to kindly apologise \' ) where > placing the adverb anywhere else would cause ambiguity . > > jennifer chew an example i once concocted to justify " splitting the infintive " ( or not , as the case may be ) is : a ) after a heavy meal , i prepared slowly to go home digesting b ) after a heavy meal , i prepared to slowly go home digesting c ) after a heavy meal , i prepared to go home slowly digesting in this context , with the possible exception of the third case , the natural ( and therefore near-enough unambiguous ) associat

## Task 2) Tokenize and remove punctuation

Now we need to split the words, a process called *tokenization* by linguists, and remove punctuation. 

We will use the Python [Natural Language Toolkit](http://www.nltk.org) *NLTK* to do the tokenization (rather than splitting ourselves, as these specialist tools usually do that better than we can ourselves). We use the NLTK function word_tokenize, see here for a code example: [http://www.nltk.org/book/ch03.html](http://www.nltk.org/book/ch03.html). 5%

Then we will remove punctuation. There is no specific funtion for this, so we use a regular expression (see here for info [https://docs.python.org/3/library/re.html?highlight=re#module-re](https://docs.python.org/3/library/re.html?highlight=re#module-re)) in a list comprehension (here's a nice visual explanation: [http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/](http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)). 5% 

We use a new technique here: we separate keys and values of the RDD, using the RDD functions `keys()` and `values()`, which yield each a new RDD. Then we process the values and *zip* them together with the keys again. See here for documentation: [http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.zip](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.zip).  We wrap the whole sequence into one function `prepareTokenRDD` for later use. 5%

In [30]:
def tokenize(text):
    """ Apply the nltk.word_tokenize() method to our text, return the token list. """
nltk.download('punkt') # this loads the standard NLTK tokenizer model 
    # it is important that this is done here in the function, as it needs to be done on every worker.
    # If we do the download outside a this function, it would only be executed on the driver     
#>>>    return ... # use the nltk function word_tokenize
nltk.word_tokenize(text)
    
def removePunctuation(tokens):
    """ Remove punctuation characters from all tokens in a provided list. """
    # this will remove all punctiation from string s: re.sub('[()\[\],.?!";_]','',s)
#>>>    tokens2 =  [re.sub('[()\[\],.?!";_]','',s) for i in s ] # use a list comprehension to remove punctuaton
    tokens2 =  [re.sub('[()\[\],.?!";_]','',s) for s in tokens ]
    return tokens2
    
def prepareTokenRDD(fn_txt_RDD):
    """ Take an RDD with (filename,text) elements and transform it into a (filename,[token ...]) RDD without punctuation characters. """

    
nltk.download('punkt')   
rdd_vals2 = rdd1.values() # It's convenient to process only the values. 
rdd_vals3 = rdd_vals2.map(nltk.word_tokenize) # Create a tokenised version of the values by mapping


#tokens2 =  [re.sub('[()\[\],.?!";_]','',s) for s in rdd_vals3 ]
#rdd_vals4 = rdd_vals3.map([re.sub('[()\[\],.?!";_]','')])# remove punctuation from the values
rdd_kv = rdd1.keys().zip(rdd_vals3)
                           
                           
                           # we zip the two RDDs together 
# i.e. produce tuples with one itme from each RDD.
# This works because we have only applied mappings to the values, 
# therefore the items in both RDDs are still aligned.
# now remove any empty value strings (i.e. length 0) that we may have created by removing punctiation.
# >>> now remove any empty strings (i.e. length 0) that we may have
# created by removing punctuation, and resulting entries without words left.
#rdd_kvr = rdd_kv.map(lambda x: (x[0], [s for s in x[1] if len(s)>0])) # remove empty strings using RDD.map and a lambda. TIP len(s) gives you the lenght of string. 
#rdd_kvrf = rdd_kvr.filter(lambda x: len(x[0]) > 0) # remove items without tokens using RDD.filter and a lambda. 
#rdd2 = rdd_kvrf    
    
    
    # >>> Question: why should this be filtering done after zipping the keys and values together?
    #
    #
    #
    # Answer : 
    # The main reason to do the filtering step after the zipping is because we need to have the same number of values and keys.
    # The filtering step should be done after the zipping because if we would have used the zipping and then the tokenization/remove-punctuation we would risk to loose the pairing keys
    # 
    # After applying tokenization and remove punctuation only in the values, and after zipping together the values and the keys, we produce tuples with one item from each RDD.
    # We can now remove empty string created after the removal of punctuation.
    #
   
    #return rdd_kvrf 

#rdd2 = prepareTokenRDD(rdd1) # Use a small RDD for testing.
#print(rdd2.take(1)) # For checking result of task 2. 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonezanetti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonezanetti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
rdd1.take(1)

[('8-1064msg1',
  'Subject: re : 8 . 1044 , disc : grammar in schools\n\n( re message from : linguist @ linguistlist . org ) > > linguist list : vol-8 - 1044 . sat jul 12 1997 . issn : 1068-4875 . > > subject : 8 . 1044 , disc : grammar in schools > > i know and teach that not all infinitives contain ` to \' . i also give > the students examples ( e . g . ` i asked him to kindly apologise \' ) where > placing the adverb anywhere else would cause ambiguity . > > jennifer chew an example i once concocted to justify " splitting the infintive " ( or not , as the case may be ) is : a ) after a heavy meal , i prepared slowly to go home digesting b ) after a heavy meal , i prepared to slowly go home digesting c ) after a heavy meal , i prepared to go home slowly digesting in this context , with the possible exception of the third case , the natural ( and therefore near-enough unambiguous ) association of the adverb is as follows : a ) after a heavy meal , i prepared _ slowly to go home digest

In [33]:
nltk.download('punkt') # Apply the Punkt sentence tokenizer: the standard tokenizer for English language

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonezanetti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
rdd_vals2 = rdd1.values() # This leaves out the key ( the filename. ex: '8-1064msg1') 

In [37]:
rdd1.values().take(1)

['Subject: re : 8 . 1044 , disc : grammar in schools\n\n( re message from : linguist @ linguistlist . org ) > > linguist list : vol-8 - 1044 . sat jul 12 1997 . issn : 1068-4875 . > > subject : 8 . 1044 , disc : grammar in schools > > i know and teach that not all infinitives contain ` to \' . i also give > the students examples ( e . g . ` i asked him to kindly apologise \' ) where > placing the adverb anywhere else would cause ambiguity . > > jennifer chew an example i once concocted to justify " splitting the infintive " ( or not , as the case may be ) is : a ) after a heavy meal , i prepared slowly to go home digesting b ) after a heavy meal , i prepared to slowly go home digesting c ) after a heavy meal , i prepared to go home slowly digesting in this context , with the possible exception of the third case , the natural ( and therefore near-enough unambiguous ) association of the adverb is as follows : a ) after a heavy meal , i prepared _ slowly to go home digesting b ) after a h

In [39]:
rdd_vals3 = rdd_vals2.map(nltk.word_tokenize)  # This tokenise the RDDs by applying work_tokenize to each of them through the map function

In [40]:
rdd_vals3.take(1)

[['Subject',
  ':',
  're',
  ':',
  '8',
  '.',
  '1044',
  ',',
  'disc',
  ':',
  'grammar',
  'in',
  'schools',
  '(',
  're',
  'message',
  'from',
  ':',
  'linguist',
  '@',
  'linguistlist',
  '.',
  'org',
  ')',
  '>',
  '>',
  'linguist',
  'list',
  ':',
  'vol-8',
  '-',
  '1044',
  '.',
  'sat',
  'jul',
  '12',
  '1997',
  '.',
  'issn',
  ':',
  '1068-4875',
  '.',
  '>',
  '>',
  'subject',
  ':',
  '8',
  '.',
  '1044',
  ',',
  'disc',
  ':',
  'grammar',
  'in',
  'schools',
  '>',
  '>',
  'i',
  'know',
  'and',
  'teach',
  'that',
  'not',
  'all',
  'infinitives',
  'contain',
  '`',
  'to',
  "'",
  '.',
  'i',
  'also',
  'give',
  '>',
  'the',
  'students',
  'examples',
  '(',
  'e',
  '.',
  'g',
  '.',
  '`',
  'i',
  'asked',
  'him',
  'to',
  'kindly',
  'apologise',
  "'",
  ')',
  'where',
  '>',
  'placing',
  'the',
  'adverb',
  'anywhere',
  'else',
  'would',
  'cause',
  'ambiguity',
  '.',
  '>',
  '>',
  'jennifer',
  'chew',
  'an',
  'ex

In [48]:
def removePunctuation(tokens):  # FIND OUT HOW TO CHANGE THIS FUNCTION !!!! 
    """ Remove punctuation characters from all tokens in a provided list. """
    # this will remove all punctiation from string s: re.sub('[()\[\],.?!";_]','',s)
#>>>    tokens2 =  [re.sub('[()\[\],.?!";_]','',s) for i in s ] # use a list comprehension to remove punctuaton
    tokens2 =  [re.sub('[()\[\],.";_]','',s) for s in tokens ]
    return tokens2




In [49]:
rdd_vals4 = rdd_vals3.map(removePunctuation) # This is helpful to remove noise from the data, i.e we remove the punctuation that does not lead
# any significant inside to the model.
# However, we keep the ! and ? as spam tend to often have these simbols as a way for advertisment to give enthusiasm



In [50]:
rdd_vals4.take(1)

[['Subject',
  ':',
  're',
  ':',
  '8',
  '',
  '1044',
  '',
  'disc',
  ':',
  'grammar',
  'in',
  'schools',
  '',
  're',
  'message',
  'from',
  ':',
  'linguist',
  '@',
  'linguistlist',
  '',
  'org',
  '',
  '>',
  '>',
  'linguist',
  'list',
  ':',
  'vol-8',
  '-',
  '1044',
  '',
  'sat',
  'jul',
  '12',
  '1997',
  '',
  'issn',
  ':',
  '1068-4875',
  '',
  '>',
  '>',
  'subject',
  ':',
  '8',
  '',
  '1044',
  '',
  'disc',
  ':',
  'grammar',
  'in',
  'schools',
  '>',
  '>',
  'i',
  'know',
  'and',
  'teach',
  'that',
  'not',
  'all',
  'infinitives',
  'contain',
  '`',
  'to',
  "'",
  '',
  'i',
  'also',
  'give',
  '>',
  'the',
  'students',
  'examples',
  '',
  'e',
  '',
  'g',
  '',
  '`',
  'i',
  'asked',
  'him',
  'to',
  'kindly',
  'apologise',
  "'",
  '',
  'where',
  '>',
  'placing',
  'the',
  'adverb',
  'anywhere',
  'else',
  'would',
  'cause',
  'ambiguity',
  '',
  '>',
  '>',
  'jennifer',
  'chew',
  'an',
  'example',
  'i',
 

In [51]:
rdd_kv = rdd1.keys().zip(rdd_vals4) # we zip the two RDDs together 
    # i.e. produce tuples with one item from each RDD.
    # This works because we have only applied mappings to the values, 
    # therefore the items in both RDDs are still aligned.
    # >>> now remove any empty strings (i.e. length 0) that we may have
    # created by removing punctuation, and resulting entries without words left.

In [52]:
rdd_kv.take(1)

[('8-1064msg1',
  ['Subject',
   ':',
   're',
   ':',
   '8',
   '',
   '1044',
   '',
   'disc',
   ':',
   'grammar',
   'in',
   'schools',
   '',
   're',
   'message',
   'from',
   ':',
   'linguist',
   '@',
   'linguistlist',
   '',
   'org',
   '',
   '>',
   '>',
   'linguist',
   'list',
   ':',
   'vol-8',
   '-',
   '1044',
   '',
   'sat',
   'jul',
   '12',
   '1997',
   '',
   'issn',
   ':',
   '1068-4875',
   '',
   '>',
   '>',
   'subject',
   ':',
   '8',
   '',
   '1044',
   '',
   'disc',
   ':',
   'grammar',
   'in',
   'schools',
   '>',
   '>',
   'i',
   'know',
   'and',
   'teach',
   'that',
   'not',
   'all',
   'infinitives',
   'contain',
   '`',
   'to',
   "'",
   '',
   'i',
   'also',
   'give',
   '>',
   'the',
   'students',
   'examples',
   '',
   'e',
   '',
   'g',
   '',
   '`',
   'i',
   'asked',
   'him',
   'to',
   'kindly',
   'apologise',
   "'",
   '',
   'where',
   '>',
   'placing',
   'the',
   'adverb',
   'anywhere',
   'els

In [53]:
rdd_kvr = rdd_kv.map(lambda x: (x[0], [s for s in x[1] if len(s)>0])) # remove empty strings using RDD.map and a lambda. TIP len(s) gives you the lenght of string. 
# S. It means that for each variable I want the x[0] ( key ) and x[1] value only if the length of the string is more than 0.
# This helps to avoid strings accidentaly created to be included

rdd_kvrf = rdd_kvr.filter(lambda x: len(x[0]) > 0) # remove items without tokens using RDD.filter and a lambda. 



In [54]:
print(rdd_kvrf.take(1))

[('8-1064msg1', ['Subject', ':', 're', ':', '8', '1044', 'disc', ':', 'grammar', 'in', 'schools', 're', 'message', 'from', ':', 'linguist', '@', 'linguistlist', 'org', '>', '>', 'linguist', 'list', ':', 'vol-8', '-', '1044', 'sat', 'jul', '12', '1997', 'issn', ':', '1068-4875', '>', '>', 'subject', ':', '8', '1044', 'disc', ':', 'grammar', 'in', 'schools', '>', '>', 'i', 'know', 'and', 'teach', 'that', 'not', 'all', 'infinitives', 'contain', '`', 'to', "'", 'i', 'also', 'give', '>', 'the', 'students', 'examples', 'e', 'g', '`', 'i', 'asked', 'him', 'to', 'kindly', 'apologise', "'", 'where', '>', 'placing', 'the', 'adverb', 'anywhere', 'else', 'would', 'cause', 'ambiguity', '>', '>', 'jennifer', 'chew', 'an', 'example', 'i', 'once', 'concocted', 'to', 'justify', '``', 'splitting', 'the', 'infintive', '``', 'or', 'not', 'as', 'the', 'case', 'may', 'be', 'is', ':', 'a', 'after', 'a', 'heavy', 'meal', 'i', 'prepared', 'slowly', 'to', 'go', 'home', 'digesting', 'b', 'after', 'a', 'heavy', '

In [55]:
print(rdd1.take(1))

[('8-1064msg1', 'Subject: re : 8 . 1044 , disc : grammar in schools\n\n( re message from : linguist @ linguistlist . org ) > > linguist list : vol-8 - 1044 . sat jul 12 1997 . issn : 1068-4875 . > > subject : 8 . 1044 , disc : grammar in schools > > i know and teach that not all infinitives contain ` to \' . i also give > the students examples ( e . g . ` i asked him to kindly apologise \' ) where > placing the adverb anywhere else would cause ambiguity . > > jennifer chew an example i once concocted to justify " splitting the infintive " ( or not , as the case may be ) is : a ) after a heavy meal , i prepared slowly to go home digesting b ) after a heavy meal , i prepared to slowly go home digesting c ) after a heavy meal , i prepared to go home slowly digesting in this context , with the possible exception of the third case , the natural ( and therefore near-enough unambiguous ) association of the adverb is as follows : a ) after a heavy meal , i prepared _ slowly to go home digestin

# STORE TEXT ( See how to call them ) Into  VECTORS of Fixed Length

Task 3) Creating normalised TF.IDF vectors of defined dimensionality, measure the effect of caching.

We use the hashing trick to create fixed size TF vectors directly from the word list now (slightly different from the previous lab, where we used *(word,count)* pairs.). Write a bit of code as needed. (5%)

Then we'll use the IDF and Normalizer functions provided by Spark. They use a slightly different pattern than RDD.map and reduce, have a look at the examples here in the documentation for Normalizer  and IDF:
[http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer), [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF) (5%)

We want control of the dimensionality in the `normTFIDF` function, so we introduce an argument into our functions that enables us to vary dimensionalty later. Here is also an opportunity to benefit from caching, i.e. persisting the RDD after use, so that it will not be recomputed.  (5%)

In [135]:
# HERE YOU CAN EXPLAIN THE CONCEPT OF VECTORIZE A WORD

#def hashing_vectorize(text,N): # arguments: the list and the size of the output vector


# hash return an encode for each word ( sequence of number and words. See after)

#example: 
text = 'Hello'
N = 10
v = [0] * N  # create vector of 0s of the dimension we define 
print('Initial vector:',v)
for word in text: # iterate through the words 
#>>>              # get the hash value
    h = hash(word)  
    print('Hashed word:', h)
    print('Hashed word % N:', h % N)
#>>>              # add 1 at the hashed address 
    v[h % N] = v[h % N] + 1 # You add one to this position of the vector
    print(v)
print('Vector identifying the Word', text, ':',v)
 # return hashed word vector


# So regardless the hash generated you may end up having same vectors for diff.
# Letters ?
# 

Initial vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Hashed word: 3563408940069010972
Hashed word % N: 2
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Hashed word: 4787531584920406966
Hashed word % N: 6
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Hashed word: -7280757250879665682
Hashed word % N: 8
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
Hashed word: -7280757250879665682
Hashed word % N: 8
[0, 0, 1, 0, 0, 0, 1, 0, 2, 0]
Hashed word: -7614204387349661840
Hashed word % N: 0
[1, 0, 1, 0, 0, 0, 1, 0, 2, 0]
Vector identifying the Word Hello : [1, 0, 1, 0, 0, 0, 1, 0, 2, 0]


In [139]:
from pyspark import StorageLevel
from pyspark.mllib.feature import IDF, Normalizer



# The MAIN GOAL IS TO HAVE RETURNED NUMERICAL INPUTS THAT CAN BE ABLE TO 
# 1. BE INPUT INTO MODELS
# 2. HAVE THE SAME VECTOR SIZE THAT I DEFINE WITH THE ARGUMENT N 

def hashing_vectorize(text,N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s of the dimension we define 
    for word in text: # iterate through the words 
#>>>              # get the hash value
        h = hash(word) # EVERY WORD OBTAIN A HASH: Look at youtube video so useful
#>>>              # add 1 at the hashed address 
        v[h % N] = v[h % N] + 1
    return v # return hashed word vector

# https://www.youtube.com/watch?v=2BldESGZKB8 Super useful to explain what this is 


testDim = 10
rdd2 = rdd_kvrf # Use a small RDD for testing.

keysRDD = rdd2.keys() # These are the labels that identify whether is spam or ham
tokensRDD = rdd2.values() # I select only the values 

tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens,testDim)) #>>> passing the vecDim value. TIP: you need a lambda.
print('Tokens before Vectorization:', tokensRDD.take(1))
print('Tokens after Vectorization:', tfVecRDD.take(1))

Tokens before Vectorization: [['Subject', ':', 're', ':', '8', '1044', 'disc', ':', 'grammar', 'in', 'schools', 're', 'message', 'from', ':', 'linguist', '@', 'linguistlist', 'org', '>', '>', 'linguist', 'list', ':', 'vol-8', '-', '1044', 'sat', 'jul', '12', '1997', 'issn', ':', '1068-4875', '>', '>', 'subject', ':', '8', '1044', 'disc', ':', 'grammar', 'in', 'schools', '>', '>', 'i', 'know', 'and', 'teach', 'that', 'not', 'all', 'infinitives', 'contain', '`', 'to', "'", 'i', 'also', 'give', '>', 'the', 'students', 'examples', 'e', 'g', '`', 'i', 'asked', 'him', 'to', 'kindly', 'apologise', "'", 'where', '>', 'placing', 'the', 'adverb', 'anywhere', 'else', 'would', 'cause', 'ambiguity', '>', '>', 'jennifer', 'chew', 'an', 'example', 'i', 'once', 'concocted', 'to', 'justify', '``', 'splitting', 'the', 'infintive', '``', 'or', 'not', 'as', 'the', 'case', 'may', 'be', 'is', ':', 'a', 'after', 'a', 'heavy', 'meal', 'i', 'prepared', 'slowly', 'to', 'go', 'home', 'digesting', 'b', 'after', '

In [ ]:
# STILL NEED TO UNDERSTAND THIS PART !

# since we will read more than once, 
# caching in Memory will make things quicker.
# https://www.quora.com/What-will-be-the-default-storage-level-for-RDD-in-Spark



tfVecRDD.persist(StorageLevel.MEMORY_ONLY) 
idf = IDF() # create IDF object
idfModel = idf.fit(tfVecRDD) # calculate IDF values
tfIdfRDD = idfModel.transform(tfVecRDD) # 2nd pass needed (see lecture slides), transforms RDD
#>>>    norm = idf... # create a Normalizer object like in the example linked above
norm = Normalizer()
#>>>    normTfIdfRDD = norm. ... # and apply it to the tfIdfRDD 
normTfIdfRDD = norm.transform(tfIdfRDD)
#>>>    zippedRDD = ... # zip the keys and values together
zippedRDD = keysRDD.zip(normTfIdfRDD)
rdd3 = zippedRDD

In [140]:
rdd3.take(1) 
# I vectorised it and normalized it, linked the keys together and check what else
# Check what is Dense Vector

[('8-1064msg1',
  DenseVector([0.6222, 0.124, 0.1883, 0.0, 0.0, 0.4141, 0.5355, 0.2159, 0.2392, 0.0]))]

In [ ]:
# It would definitely make sense to define everything within a function 
# which will make sense when changing and defining the Vector dimension

# Define the labels

 Task 4) Create LabeledPoints 

Determine whether the file is spam (i.e. the filename contains ’spmsg’) and replace the filename by a 1 (spam) or 0 (non-spam) accordingly. Use `RDD.map()` to create an RDD of LabeledPoint objects. See here [http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression](http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression) for an example, and here [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint) for the `LabeledPoint` documentation. (10%)

There is a handy function of Python strings called startswith: e.g. 'abc'.startswith('ab) will return true. The relevant Python syntax here is a conditional expression: **``<a> if <yourCondition> else <b>``**, i.e. 1 if the filename starts with 'spmsg' and otherwise 0.

In [145]:
from pyspark.mllib.regression import LabeledPoint

print(rdd3.take(1)) 

cls_vec_RDD = rdd3.map(lambda x: (1 if x[0].startswith('spm') else 0, x[1]))
  
# now we can create the LabeledPoint objects with (class,vector) arguments
# It creates a LabeledPoint <- I don't know why: You need to check why it does this. 
lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0],cls_vec[1]) ) 

print(lp_RDD.take(1))    


[('8-1064msg1', DenseVector([0.6222, 0.124, 0.1883, 0.0, 0.0, 0.4141, 0.5355, 0.2159, 0.2392, 0.0]))]
[LabeledPoint(0.0, [0.6221689348917344,0.12400296977829081,0.18830080595962678,0.0,0.0,0.4140596001139784,0.5355363340306855,0.21585702146591362,0.23923443562140975,0.0])]


# PreProcessing 

Task 5) Complete the preprocessing 

It will be useful to have a single function to do the preprocessing. So integrate everything here. (5%)

In [ ]:
# Here it wraps everything up and define function to be able to import new data super quickly 
# and eventually change the size of the Vector

# -> I think you need to do everything again by including TRAIN and TEST A

## Task 6) Train some classifiers 

Use the `LabeledPoint` objects to train a classifier, specifically the *LogisticRegression*, *Naive Bayes*, and *Support Vector Machine*. Calculate the accuracy of the model on the training set (again, follow this example [http://spark.apache.org/docs/2.1.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) and here is the documentation for the classifiers [LogisticRegressionWithLBFGS](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS), [NaiveBayes](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.NaiveBayes), [SVMWithSGD](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMWithSGD).  (10%) 